# Kolla nya idéer och features 
### Speciellt för att prioritera mellan hästar i olika lopp
- kelly vs proba
- antal hästar i loppen som feature
- motståndarinfo
  - De tre bäst rankade motståndarna i loppet, deras antal streck
  - genomsnittet av någon ranking för hästarna
  - ...




## 1. En sorterad lista med proba och/eller kelly 
Från den listan plockas hästar tills max insats är nådd, kanske lltid spika en eller två hästar

In [3]:
# moduler
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
import flaml
import pickle
from IPython.display import display
# # read a model from a file with pickle
with open('..\\modeller\\FLAML_model.sav', 'rb') as f:
    model = pickle.load(f)
with open('..\\modeller\\FLAML2_model.sav', 'rb') as f:
    model2 = pickle.load(f)
    
pd.set_option('display.width', 100)


In [4]:
def remove_features(df_, remove_mer=[]):
    df = df_.copy()
    df.drop([ 'startnr', 'avd','vodds', 'podds', 'bins', 'h1_dat',
              'h2_dat', 'h3_dat', 'h4_dat', 'h5_dat'], axis=1, inplace=True)
    if remove_mer:
            df.drop(remove_mer, axis=1, inplace=True)

    return df


def load_enc():
    with open('..\\modeller/encoder.sav', 'rb') as file:
        enc = pickle.load(file)
    return enc
def ordinal_enc(df_, feature):
    enc = load_enc()
    df = df_.copy()
    enc.transform(df[[feature]])
    # enc.fit_transform(df[[feature]])
    df[[feature]] = enc.transform(df[[feature]])
    return df

In [49]:
# skapa data
df = pd.read_csv('..\\all_data.csv')
X=df.copy()
X.drop('plac', axis=1, inplace=True)
X = ordinal_enc(X, 'häst')
y=(df.plac==1)*1   # plac 1 eller 0


In [50]:
# model1 är FLAML och model2 är FLAML2
proba = model.predict_proba(remove_features(X))
proba2 = model2.predict_proba(remove_features(X))
X['proba'] = proba[:,1]
X['proba2'] = proba2[:, 1]


In [51]:
def kelly(proba, streck, odds):  # proba = prob winning, streck i % = streck
    if odds is None:
        o = 100/streck.copy()
    else:
        o=odds.copy()
            
    # for each values > 50 in odds set to 1
    o[o>40]=1
    return (o*proba - (1-proba))/o
            

Xtst = pd.read_csv('..\\sparad_scrape.csv')

#df['f'] = (df.proba*df.vodds - 1) / (df.vodds-1)  # kelly formel
Xtst['kellys'] = kelly(Xtst.proba, Xtst.streck+1, None)
proba2 = model2.predict_proba(remove_features(Xtst))
Xtst['proba2'] = proba2[:, 1]
Xtst['kelly2'] = kelly(Xtst.proba2, Xtst.streck+1, None)


In [52]:
Xtst[['avd' , 'startnr', 'proba','kellys','proba2','kelly2']]

,avd,startnr,proba,kellys,proba2,kelly2
0,1,2,0.407517,0.152750,0.259113,-0.059468
1,1,6,0.134074,-0.030452,0.289233,0.154187
2,1,5,0.122549,-0.044167,0.228171,0.081523
3,1,11,0.054934,-0.011221,0.098867,0.035787
4,1,8,0.052131,-0.014220,0.049168,-0.017391
...,...,...,...,...,...,...
78,7,1,0.036045,0.007126,0.048821,0.020285
79,7,11,0.034413,-0.931174,0.042815,-0.914370
80,7,7,0.033703,-0.932594,0.051732,-0.896537
81,7,8,0.033485,-0.933031,0.036655,-0.926689


## plocka hästar efter proba och kelly till en df
1. Upp till 300:-
2. Minst en häst per avd
3. Lopp utan favoriter får utspridda låga värden och väljs inte för "helgardering" - hur lösa detta?
   

In [53]:
def compute_total_insats(df):
    insats=0
    # group by avd
    summa = df.groupby('avd').avd.count().prod() / 2   
    return summa

def fill_up_dfSpel(df_, max_insats=300,metod='proba'):
    # dfSpel är de hästar som ska spela
    df=df_.copy()
    dfSpel = pd.DataFrame()
    df=df.sort_values(by=metod, ascending=[False])
    curr_insats = 0
    for index, row in df.iterrows():
        dfSpel = dfSpel.append(row)
        curr_insats = compute_total_insats(dfSpel)
        if curr_insats>max_insats+30:
            dfSpel = dfSpel.iloc[:-1,:]
            break
        # print(curr_insats)

    print(compute_total_insats(dfSpel))
    return dfSpel


## Lägg in antal hästar som feature

In [54]:
def lägg_in_antal_hästar(df_):
    df = df_.copy()
    # count number per avdeling
    # df_count = df.groupby(['datum', 'avd'])['avd'].count()
    df['ant_per_lopp'] = None
    df['ant_per_lopp'] = df.groupby(['datum', 'avd'])['avd'].transform('count')

    # drop startnr because it has too many NaN
    # drop vodds, podds, bins because they are not known before v75
    X = df.drop(['plac', 'startnr', 'vodds', 'podds', 'bins',   # behåll avd som tas bort senare
                'h1_dat', 'h2_dat', 'h3_dat', 'h4_dat', 'h5_dat', ], axis=1)

    y = (df['plac'] == 1)*1

    return X, y
X,y = lägg_in_antal_hästar(df)

In [55]:
X.columns

Index(['datum', 'avd', 'bana', 'häst', 'kusk', 'streck', 'kr', 'spår', 'dist', 'lopp_dist',
       'start', 'ålder', 'kön', 'pris', 'h1_kusk', 'h1_bana', 'h1_spår', 'h1_plac', 'h1_pris',
       'h1_odds', 'h1_kmtid', 'h2_kusk', 'h2_bana', 'h2_spår', 'h2_plac', 'h2_pris', 'h2_odds',
       'h2_kmtid', 'h3_kusk', 'h3_bana', 'h3_spår', 'h3_plac', 'h3_pris', 'h3_odds', 'h3_kmtid',
       'h4_kusk', 'h4_bana', 'h4_spår', 'h4_plac', 'h4_pris', 'h4_odds', 'h4_kmtid', 'h5_kusk',
       'h5_bana', 'h5_spår', 'h5_plac', 'h5_pris', 'h5_odds', 'h5_kmtid', 'h1_dist', 'h2_dist',
       'h3_dist', 'h4_dist', 'h5_dist', 'h1_auto', 'h2_auto', 'h3_auto', 'h4_auto', 'h5_auto',
       'h1_perf', 'h2_perf', 'h3_perf', 'h4_perf', 'h5_perf', 'senast', 'delta1', 'delta2',
       'delta3', 'delta4', 'ant_per_lopp'],
      dtype='object')

In [56]:
# använda catboostClassifier
from catboost import CatBoostClassifier, Pool,cv


In [57]:
# get numerical features and cat_features
num_features = list(X.drop('avd', axis=1).select_dtypes(include=[np.number]).columns)
cat_features = list(X.drop('avd', axis=1).select_dtypes(include=['object']).columns)
print(f'Rätt längd på summan (med datum)? {1+len(num_features)+len(cat_features)} == {len(X.columns)}')
# check cat_features isna
print('NaN before:', X[cat_features].isna().sum()[
      X[cat_features].isna().sum() > 0].sort_values(ascending=False).sum())

# impute 'missing' for all NaN in cat_features
X[cat_features] = X[cat_features].fillna('missing')
print('NaN after:', X[cat_features].isna().sum().sum())


Rätt längd på summan (med datum)? 70 == 70
NaN before: 246
NaN after: 0


In [58]:
"""
cross validation with catboost
"""
def catboost_cv(X, y, cat_features, fold_count=5):

    # Create a pool
    pool = Pool(data=X, label=y, cat_features=cat_features)
    params = {"iterations": 1000,
              "loss_function": "Logloss",
              "verbose": False}

    cv_results = cv(pool, params, fold_count=fold_count,

                    early_stopping_rounds=100,
                    # loss_function='Logloss',
                    shuffle=False,
                    type='TimeSeries',
                    verbose=False)
    return cv_results


In [59]:
result = catboost_cv(X.drop('avd', axis=1), y, cat_features)

Training on fold [0/5]

bestTest = 0.237182333
bestIteration = 137

Training on fold [1/5]

bestTest = 0.2386943467
bestIteration = 117

Training on fold [2/5]

bestTest = 0.2446307386
bestIteration = 161

Training on fold [3/5]

bestTest = 0.2356321447
bestIteration = 138

Training on fold [4/5]

bestTest = 0.2374105316
bestIteration = 184



In [60]:
result['test-Logloss-mean'].min()


0.23890538767163833

med  antal hästar som feature: 0.2396187999030749  
utan antal hästar som feature: 0.23964830518637106  
Minimalt bättre men det är inte vad jag vill uppnå utan bättre viktning mellan hästar i olika lopp

# Kör tester med olika lösningar 

In [5]:
# skapa data
df = pd.read_csv('..\\all_data.csv')
X = df.copy()
X.drop('plac', axis=1, inplace=True)
# X = ordinal_enc(X, 'häst')
y = (df.plac == 1)*1   # plac 1 eller 0


def get_data_for_catboost(X):
    # get numerical features and cat_features
    num_features = list(X.drop('avd', axis=1).select_dtypes(
        include=[np.number]).columns)
    cat_features = list(X.drop('avd', axis=1).select_dtypes(
        include=['object']).columns)
    print(f'Rätt längd på summan (med datum)? {1+len(num_features)+len(cat_features)} == {len(X.columns)}')
    # check cat_features isna
    print('NaN before:', X[cat_features].isna().sum()[X[cat_features].isna().sum() > 0].sort_values(ascending=False).sum())

    # impute 'missing' for all NaN in cat_features
    X[cat_features] = X[cat_features].fillna('missing')
    print('NaN after:', X[cat_features].isna().sum().sum())
    return X, cat_features

X, cat_features = get_data_for_catboost(X)
# cat_features = list(X.drop('avd', axis=1).select_dtypes(include=['object']).columns)
print('cat_features:',cat_features)


def lägg_in_antal_hästar(df_):
    df = df_.copy()
    df['ant_per_lopp'] = None
    df['ant_per_lopp'] = df.groupby(['datum', 'avd'])['avd'].transform('count')
    return df

# Skapa X_train och X_test 
def create_X_train_test(X, y, datum):
    X_train = X[X.datum < datum]
    X_test = X[X.datum == datum]
    y_train = y[X.datum < datum]
    y_test = y[X.datum == datum]
    return X_train, X_test, y_train, y_test

#make häst, bana, kusk lower case

for f in ['häst', 'bana', 'kusk', 'h1_kusk', 'h2_kusk', 'h3_kusk', 'h4_kusk', 'h5_kusk', 'h1_bana', 'h2_bana', 'h3_bana', 'h4_bana', 'h5_bana']:
    # print(X[f], type(X[f]))
    X[f] = X[f].str.lower()


Rätt längd på summan (med datum)? 78 == 78
NaN before: 369
NaN after: 0
cat_features: ['datum', 'bana', 'häst', 'kusk', 'kön', 'h1_dat', 'h1_kusk', 'h1_bana', 'h2_dat', 'h2_kusk', 'h2_bana', 'h3_dat', 'h3_kusk', 'h3_bana', 'h4_dat', 'h4_kusk', 'h4_bana', 'h5_dat', 'h5_kusk', 'h5_bana']


In [6]:
def kelly(proba, streck, odds):  # proba = prob winning, streck i % = streck
    if odds is None:
        o = 100/streck.copy()
    else:
        o = odds.copy()

    # for each values > 50 in odds set to 1
    o[o > 40] = 1
    return (o*proba - (1-proba))/o

In [7]:
# pick the häst to play
def compute_total_insats(df):
    insats = 0
    # group by avd
    summa = df.groupby('avd').avd.count().prod() / 2
    return summa

def hitta_största_diff_per_avd(df_, ant):
    df=df_.copy()
    # sortera på avd,proba
    df = df.sort_values(['avd','streck'], ascending=[False, False])
    diff_list = []
    for avd in range(1,8):
        diff = df.loc[df.avd==avd].streck.iloc[0] - df.loc[df.avd==avd].streck.iloc[1]
        diff_list.append((avd,diff))
        
     # sortera på diff
    diff_list = sorted(diff_list, key=lambda x: x[1], reverse=True)   
    return diff_list[:ant]

# temp is a list of tuples. Each tuple is (avd, diff). check if avd is in the list
def check_avd(avd, temp):
    for t in temp:
        if t[0] == avd:
            return True
    return False   


In [8]:
  
def spela(df_, max_insats=300, proba=True, kelly=True,ant_favoriter=0):
    df = df_.copy()
    # print('omg df',df.shape)
    dfSpel = pd.DataFrame()
    if proba:
        df = df.sort_values(by='proba', ascending=False)
        if kelly:
            df2 = df.sort_values(by='kelly', ascending=False)
    else:
        df = df.sort_values(by='kelly', ascending=False) # must be kelly
        
    # se till att vi har för alla avd. Välj den bästa per avd
    if proba:
        temp = df.sort_values(by=['proba'], ascending=False)
    else:
        temp = df.sort_values(by=['kelly'], ascending=False)    
    for avd in range(1, 8):
        dfSpel = dfSpel.append(temp[temp.avd == avd].iloc[0])
    #
    favorit_list = hitta_största_diff_per_avd(df, ant_favoriter)
    curr_insats = 0
    cnt=0
    for index, row in df.iterrows():
        if check_avd(row.avd, favorit_list): # Inga fler hästar för avd med en favorit
            continue
        
        dfSpel = dfSpel.append(row)
        # print(dfSpel.shape)
        # print('row',row.shape)
        curr_insats = compute_total_insats(dfSpel)
        if curr_insats > max_insats+30:
            dfSpel = dfSpel.iloc[:-1, :]
            break
        if kelly & proba:
            row2 = df2.iloc[cnt]
            # print('row2',row2.shape)
            dfSpel = dfSpel.append(row2)
            # remove duplicates in dfSpel nd keep the first
        dfSpel = dfSpel.drop_duplicates(subset=['avd', 'häst'], keep='first')
        # print('dfSpel',dfSpel.shape)    
        curr_insats = compute_total_insats(dfSpel)
        if curr_insats > max_insats+30:
            dfSpel = dfSpel.iloc[:-1, :]
            break
        cnt +=1
                
    return dfSpel

# train with curr_datum
def learn(X_train, y_train, X_test, y_test, cat_features):
    train_pool = Pool(X_train.drop('avd', axis=1), label=y_train, cat_features=cat_features)
    test_pool = Pool(X_test.drop('avd', axis=1), label=y_test, cat_features=cat_features)
    cbc = CatBoostClassifier(iterations=1000, loss_function='Logloss', eval_metric='AUC', verbose=False)
    cbc.fit(train_pool, eval_set=test_pool, early_stopping_rounds=100,use_best_model=True, verbose=False)
    return cbc

def beräkna_utdelning(dfSpel,ant_rätt, df_utdelning):
    if ant_rätt==7:
        slista = list(dfSpel.groupby('avd').avd.count()-1)
        sjuor = 1
        sexor = sum(slista)
        femmor =0
        for avd_ix in range(0, 7):
            for avd2_ix in range(avd_ix+1, 7):
                femmor += slista[avd_ix] * slista[avd2_ix]
    
    elif ant_rätt==6:
        sjuor=0
        avd_list = list(dfSpel.loc[dfSpel.rätt == 'OK']['avd'])  # avd med rätt
        missing = [x for x in range(1, 8) if x not in avd_list]
        sexor = dfSpel.loc[dfSpel.avd == missing[0], 'avd'].count()
        femmor=0
        for avd in avd_list:
            femmor+=dfSpel.loc[dfSpel.avd==avd, 'avd'].count()
        femmor=(femmor-6) * sexor
        
    elif ant_rätt==5:
        sjuor,sexor=0,0
        avd_list = list(dfSpel.loc[dfSpel.rätt == 'OK']['avd'])  # avd med rätt
        missing = [x for x in range(1, 8) if x not in avd_list]
        femmor=dfSpel.loc[dfSpel.avd==missing[0], 'avd'].count() + dfSpel.loc[dfSpel.avd==missing[1], 'avd'].count()
    else:
        sjuor,sexor,femmor = 0,0,0 
           
    datum = dfSpel.iloc[0].datum       
    priser = list(df_utdelning.loc[df_utdelning.datum ==datum, ['7rätt', '6rätt', '5rätt']].values[0])
    return sjuor,sexor,femmor, priser[0]*sjuor+ priser[1]*sexor + priser[2]*femmor

def rätta(y_validate, dfSpel, df_utdelning):
    yindex = y_validate.loc[y == 1].index
    # print(1,_,dfSpel.shape)
    ant_rätt = len(dfSpel.loc[dfSpel.index.isin(yindex)])
    # print(f': total insats {compute_total_insats(dfSpel)} med {dfSpel.shape[0]} spelade hästar: {ant_rätt} rätt')
    dfSpel['rätt'] = ' '
    dfSpel[['avd', 'streck','rätt']]
    dfSpel.loc[dfSpel.index.isin(yindex), 'rätt'] = 'OK'
    # print(2,_,dfSpel.shape)
    return  beräkna_utdelning(dfSpel, ant_rätt, df_utdelning)

def testa_flera_omg(X_,y_,typ, max_cnt=60): 
    X = X_.copy()
    y = y_.copy()

    max_insats = stats_typer.loc[stats_typer.typ == typ].insats.values[0]
    print('max_insats',max_insats)
    bProba = stats_typer.loc[stats_typer.typ == typ].proba.values[0]
    print('Proba',bProba)
    bKelly = stats_typer.loc[stats_typer.typ == typ].kelly.values[0]
    print('Kelly',bKelly)
    bAnt_hästar = stats_typer.loc[stats_typer.typ == typ].ant_hästar.values[0]
    ant_favoriter = stats_typer.loc[stats_typer.typ == typ].favoriter.values[0]
    print('ant_favoriter',ant_favoriter)
    
    if bAnt_hästar:
        X = lägg_in_antal_hästar(X)
        print('lägg_in_antal_hästar per avd')
        
    df_utdelning = pd.read_csv('..\\utdelning.csv')
    datum = df.datum.unique()
    spel_dict = {'typ':[],'datum':[],'insats':[], 'utdelning':[], 'sjuor':[], 'sexor':[], 'femmor':[]}
    kassa=0
    ix = len(datum)//2+100
    for cnt,curr_datum in enumerate(datum[ix:-1]):

        # select data
        X_train = X.loc[df.datum<curr_datum].copy()
        y_train = y.loc[df.datum<curr_datum].copy()
        X_test = X.loc[df.datum>curr_datum].copy()
        y_test = y.loc[df.datum>curr_datum].copy()
        X_validate = X.loc[df.datum==curr_datum].copy()
        y_validate = y.loc[df.datum==curr_datum].copy()
        
        # Learn
        cbc=learn(X_train, y_train, X_test, y_test, cat_features)
        
        # predict
        validate_pool = Pool(X_validate.drop('avd', axis=1),label=y_validate, cat_features=cat_features)
        X_validate['proba'] = cbc.predict_proba(validate_pool)[:, 1]
        X_validate['kelly'] = kelly(X_validate.proba, X_validate.streck+1, None)
        
        # Spela
        dfSpel = spela(X_validate, max_insats=max_insats, proba=bProba, kelly=bKelly, ant_favoriter=ant_favoriter)
        
        # Rätta
        sjuor, sexor, femmor, utdelning = rätta(y_validate, dfSpel, df_utdelning)
        insats = compute_total_insats(dfSpel) 
        ant_spelade = dfSpel.shape[0]
        kassa += utdelning-insats
        print(f'{curr_datum} - insats: {insats}, ant spel: {ant_spelade}, rätt=({sjuor},{sexor},{femmor}), Utd: {utdelning}  :\tkassa: {kassa}')
        # append insats, utdelning, sjuor to spel_dict['datum']
        spel_dict['typ'].append(typ)
        spel_dict['datum'].append(curr_datum)
        spel_dict['insats'].append(insats)
        spel_dict['utdelning'].append(utdelning)
        spel_dict['sjuor'].append(sjuor)
        spel_dict['sexor'].append(sexor)
        spel_dict['femmor'].append(femmor)
        
        if cnt+1>=max_cnt:
            break
    print('Finished')
    return spel_dict,dfSpel


stats_typer = pd.DataFrame(columns=['typ', 'ant_datum', 'insats', 'ant_hästar',
                           'proba', 'kelly', 'motst_info', 'favoriter', 'många_om_osäker'])
stats_typer.loc[0] = ['1', 200, 300, True,  True, True, False, 0, False]
stats_typer.loc[1] = ['2', 200, 300, True,  True, False, False, 0, False]
stats_typer.loc[2] = ['3', 200, 300, True,  False, True, False, 0, False]
stats_typer.loc[3] = ['4', 200, 300, False, True, True, False, 0, False]
stats_typer.loc[4] = ['5', 200, 300, False, True, False, False, 0, False]
stats_typer.loc[5] = ['6', 200, 300, False, False, True, False, 0, False]
stats_typer.loc[6] = ['7', 200, 3000, True,  True, True, False, 0, False]
stats_typer.loc[7] = ['8', 200, 300, True,  True, True, False, 2, False]
stats_typer


,typ,ant_datum,insats,ant_hästar,proba,kelly,motst_info,favoriter,många_om_osäker
0,1,200,300,True,True,True,False,0,False
1,2,200,300,True,True,False,False,0,False
2,3,200,300,True,False,True,False,0,False
3,4,200,300,False,True,True,False,0,False
4,5,200,300,False,True,False,False,0,False
5,6,200,300,False,False,True,False,0,False
6,7,200,3000,True,True,True,False,0,False
7,8,200,300,True,True,True,False,2,False


In [9]:
spel_dict, dfSpel = testa_flera_omg(X,y, '4', 200)
temp_stats = pd.DataFrame(spel_dict)
temp_stats.typ=temp_stats.typ.astype('string')
temp_stats.tail(10)

max_insats 300
Proba True
Kelly True
ant_favoriter 0
2019-08-21 - insats: 324.0, ant spel: 19, rätt=(0,0,0), Utd: 0  :	kassa: -324.0
2019-08-22 - insats: 288.0, ant spel: 18, rätt=(0,0,0), Utd: 0  :	kassa: -612.0
2019-08-23 - insats: 180.0, ant spel: 18, rätt=(0,0,0), Utd: 0  :	kassa: -792.0
2019-08-24 - insats: 288.0, ant spel: 19, rätt=(0,0,4), Utd: 828  :	kassa: -252.0
2019-08-25 - insats: 324.0, ant spel: 18, rätt=(0,0,5), Utd: 0  :	kassa: -576.0
2019-08-31 - insats: 288.0, ant spel: 19, rätt=(0,0,4), Utd: 84  :	kassa: -780.0
2019-09-01 - insats: 324.0, ant spel: 18, rätt=(0,0,0), Utd: 0  :	kassa: -1104.0
2019-09-07 - insats: 288.0, ant spel: 18, rätt=(0,2,20), Utd: 812  :	kassa: -580.0
2019-09-08 - insats: 324.0, ant spel: 19, rätt=(0,0,0), Utd: 0  :	kassa: -904.0
2019-09-14 - insats: 324.0, ant spel: 18, rätt=(0,0,5), Utd: 100  :	kassa: -1128.0
2019-09-21 - insats: 288.0, ant spel: 19, rätt=(0,0,0), Utd: 0  :	kassa: -1416.0
2019-09-28 - insats: 288.0, ant spel: 19, rätt=(0,0,6), 

,typ,datum,insats,utdelning,sjuor,sexor,femmor
158,4,2021-12-04,192.0,4236,0,4,32
159,4,2021-12-11,288.0,0,0,0,0
160,4,2021-12-18,324.0,75,0,0,5
161,4,2021-12-23,180.0,0,0,0,0
162,4,2021-12-31,324.0,0,0,0,0
163,4,2022-01-01,324.0,192,0,0,6
164,4,2022-01-08,324.0,0,0,0,0
165,4,2022-01-15,288.0,0,0,0,3
166,4,2022-01-22,270.0,336,0,0,7
167,4,2022-01-29,216.0,0,0,0,0


In [10]:
spel_stats = pd.read_csv('spel_stats.csv')
spel_stats = pd.concat([spel_stats, temp_stats], ignore_index=True)
spel_stats.typ = spel_stats.typ.astype('string')
spel_stats.drop_duplicates(subset=['typ', 'datum'], keep='last', inplace=True)
spel_stats.to_csv('spel_stats.csv', index=False)


In [11]:
def stats_for_typ(typ,stats):
    stats_row = stats.loc[stats.typ == typ]
    print(stats_row[['insats','utdelning','sjuor','sexor','femmor']].sum())
    print(f'Kassa (typ={typ}):',stats_row.utdelning.sum()-stats_row.insats.sum())
stats_for_typ('4',spel_stats)

def bästa_kassa(stats):
    typlista=[]
    for typ in ['1','2','3','4','5','6','7','8']:
        row = stats.loc[stats.typ==typ]
        kassa=row.utdelning.sum()-row.insats.sum()
        typlista.append((typ,kassa))
    # sort a list descending
    typlista.sort(key=lambda x: x[1], reverse=True)
    
    print(typlista)
bästa_kassa(spel_stats)
spel_stats

insats       48364.0
utdelning    38231.0
sjuor            7.0
sexor          155.0
femmor        1321.0
dtype: float64
Kassa (typ=4): -10133.0
[('1', 15146.0), ('8', 10856.0), ('5', 4014.0), ('3', -8790.5), ('4', -10133.0), ('6', -10157.0), ('2', -12006.0), ('7', -50076.0)]


,typ,datum,insats,utdelning,sjuor,sexor,femmor
102,7,2019-08-21,2304.0,175,0,0,7
103,7,2019-08-22,2592.0,608,0,0,8
104,7,2019-08-23,2700.0,550,0,0,5
105,7,2019-08-24,2400.0,1242,0,0,6
106,7,2019-08-25,2592.0,126,0,3,45
...,...,...,...,...,...,...,...
1175,4,2022-01-01,324.0,192,0,0,6
1176,4,2022-01-08,324.0,0,0,0,0
1177,4,2022-01-15,288.0,0,0,0,3
1178,4,2022-01-22,270.0,336,0,0,7
